### Task 29.	2x2 + 7xy + 5x2 – 11xy + 3y2

In [8]:
class Operation():                  
    
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes # The list of input nodes
        self.output_nodes = []
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
        
    def compute(self):
        pass

In [9]:
class add(Operation):
    
    def __init__(self, x, y):
         
        super().__init__([x, y])

    def compute(self, x_var, y_var):
         
        self.inputs = [x_var, y_var]
        return x_var + y_var
    
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var
    
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)
        
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
         
        _default_graph.variables.append(self)

In [10]:
class Graph():
    
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        """
        Sets this Graph instance as the Global Default Graph
        """
        global _default_graph
        _default_graph = self

In [11]:
graph = Graph()
graph.set_as_default()
a = Variable(2)
b = Variable(7)
c = Variable(5)
d = Variable(-11)
e = Variable(3)
x = Placeholder()
y = Placeholder()

f = multiply(x, x) #x2
g = multiply(a, f) #2x2
h = multiply(x, y) #xy
i = multiply(b, h) #7xy
j = add(g, i)      #2x2 + 7xy
k = multiply(c, f) #5x2
l = add(j, k)      #2x2 + 7xy + 5x2
m = multiply(d, h) #-11xy
n = add(l, m)      #2x2 + 7xy + 5x2 - 11xy
o = multiply(y, y) #y2
p = multiply(e, o) #3y2
q = add(n, p)      #2x2 + 7xy + 5x2 - 11xy + 3y2

In [12]:
import numpy as np

def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder


class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
       
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output

In [13]:
session = Session()
res = session.run(operation=q, feed_dict = {x:[1, 10, 15], y:[2, 6, 8]})

In [14]:
res

array([  11,  568, 1287])

In [15]:
2*(1*1) + 7*(1*2) + 5*(1*1) - 11*(1*2) + 3*(2*2)

11

In [ ]:
2*(10*10) + 7*(10*6) + 5*(10*1) - 11*(1*2) + 3*(2*2)